
# Jour 2 — Courbes de taux : Discount Factor, Zero Curve et Forward Rate

Ce notebook te donne tous les outils **pratiques** pour comprendre et manipuler :
- Les **discount factors** (facteurs d’actualisation)
- La **courbe de taux zéro-coupon**
- Les **taux forward** (taux implicites entre deux dates)
- Application directe au pricing Rates et à l’oral d’entretien Quant Dev.

---

## 1. Principes de base

- **Courbe de taux zéro** : pour chaque maturité, le taux d’intérêt annualisé, applicable à un investissement de “aujourd’hui à t”, sans coupon intermédiaire (ex : obligations zéro-coupon).
- **Discount factor (DF)** : combien vaut aujourd’hui 1 euro (ou 1 USD) que je recevrai à la date t ?
    - Plus le taux est haut, plus DF est bas.
    - Si r = 3% (base continue), DF(2 ans) = exp(-0.03 * 2)
- **Forward rate** : taux implicite entre deux dates (t1, t2), calculé à partir de la courbe zéro. Il répond à la question : “si je peux emprunter jusqu’à t1, puis placer de t1 à t2, quel taux dois-je obtenir entre t1 et t2 pour égaler un investissement de 0 à t2 directement ?”

### **Formules clés :**
- **Discount factor** : DF(t) = exp(-r * t), r = taux zero annualisé base continue
- **Forward rate (base continue)** : 
    - \( f_{t_1, t_2} = -\frac{\ln(DF(t_2)/DF(t_1))}{t_2-t_1} \)

---


## 2. Construisons une mini courbe de taux zéro fictive

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

years = np.array([1, 2, 3])
zero_rates = np.array([0.02, 0.025, 0.03])  # 2%, 2.5%, 3%

discount_factors = np.exp(-zero_rates * years)

for t, r, d in zip(years, zero_rates, discount_factors):
    print(f"DF({t}Y) avec r={r*100:.2f}% : {d:.5f}")

courbe = pd.DataFrame({
    "Maturity (Y)": years,
    "ZeroRate (%)": zero_rates * 100,
    "DiscountFactor": discount_factors
})
courbe



### 📌 **À retenir sur le Discount Factor**

- DF(t) < 1 pour tout t > 0.
- Plus la maturité ou le taux augmente, plus DF diminue.
- **Interprétation pratique** : pour “discounter” un cashflow, on multiplie son montant futur par DF(t).

**Exemple** : Recevoir 100€ dans 2 ans à 2.5% (base continue) vaut aujourd’hui :  
100 × exp(-0.025×2) ≈ 95.06€



## 3. Calcul du taux forward entre deux dates

**Formule base continue :**

\[ f_{t_1, t_2} = -\frac{\ln\left(\frac{DF(t_2)}{DF(t_1)}\right)}{t_2-t_1} \]

- **Sens** : quel taux garantis-je entre t1 et t2, sachant les taux zéro à t1 et t2 ?



In [ ]:

df_1 = discount_factors[0]
df_2 = discount_factors[1]
t1 = years[0]
t2 = years[1]

forward_1_2 = -np.log(df_2/df_1)/(t2-t1)
print(f"Taux forward implicite 1Y→2Y = {forward_1_2*100:.3f}%")



**Remarque** : Le forward rate n’est PAS forcément entre le zero 1Y et zero 2Y. Il dépend du “pente” de la courbe.

- Si la courbe est pentue (= taux 2Y > 1Y), le forward est plus haut que le zero 1Y.
- Cela sert à “lire le marché” : anticipations de taux futurs, pricing swap, etc.



### 🧠 **Exercices de réflexion**

1. Si le taux zéro 3Y augmente, que fait le DF(3Y) ?
2. Pourquoi le forward rate peut-il être supérieur au zero rate ?
3. Calcule le DF(4Y) si le taux zéro 4Y est 3.2% (base continue).

**Réponses orales** :  
1. DF(3Y) baisse, car plus le taux est élevé plus l’actualisation “mange” la valeur future.  
2. Si la courbe monte fortement (taux 2Y << 3Y), le forward de 2Y à 3Y sera > taux zéro 2Y.  
3. DF(4Y) = exp(-0.032 × 4) ≈ 0.8776


## 4. Fonction Python générique pour taux forward

In [ ]:

def forward_rate(df1, df2, t1, t2):
    """Calcule le taux forward continu entre t1 et t2 à partir des DF."""
    return -np.log(df2/df1)/(t2-t1)

print("Forward 2Y→3Y :", forward_rate(discount_factors[1], discount_factors[2], years[1], years[2]) * 100, "%")



## 5. (Optionnel) Interpolation de la courbe

Souvent, on ne connaît la courbe zéro qu’à quelques points (ex : 1Y, 2Y, 5Y, 10Y…).  
Pour obtenir un taux à 1.5Y, on interpole.

En pratique : linéaire, log-linéaire, cubic spline…



In [ ]:

from scipy.interpolate import interp1d

# Suppose taux à 1Y, 2Y, 3Y ; on veut le taux à 1.5Y
interp_zero = interp1d(years, zero_rates, kind="linear")
zero_1_5Y = float(interp_zero(1.5))
print(f"Taux zéro 1.5Y interpolé (linéaire) : {zero_1_5Y*100:.3f}%")

# DF interpolé
df_1_5Y = np.exp(-zero_1_5Y * 1.5)
print(f"DF(1.5Y) interpolé : {df_1_5Y:.5f}")



## 6. Visualisation de la courbe (zero + DF)

La courbe zéro est souvent croissante (structure de terme), la courbe DF toujours décroissante.



In [ ]:

plt.figure(figsize=(8,4))
plt.plot(years, zero_rates*100, marker="o", label="Zero Rate (%)")
plt.plot(years, discount_factors, marker="x", label="Discount Factor")
plt.xlabel("Maturité (années)")
plt.title("Courbe de taux zéro & Discount Factor")
plt.legend()
plt.grid()
plt.show()



---
## 📝 **À retenir pour l’entretien (fiche orale)**

- **Discount factor** : combien vaut 1€ dans t ans ? C’est exp(-r * t) si r = taux zéro annualisé.
- **Forward rate** : taux implicite entre deux dates, lisible sur la courbe zéro. Sert à anticiper/piloter des swaps.
- Si courbe zéro donnée sous forme de points, il faut interpoler pour utiliser/pricer précisément.
- Pour tout pricing en Rates, on actualise les cashflows futurs par leur DF correspondant.
- Savoir expliquer ce tableau :

| Année | Taux zéro | DF   |
|-------|-----------|------|
|   1   |   2%      | 0.9802 |
|   2   |   2.5%    | 0.9512 |
|   3   |   3%      | 0.9131 |

---

**Entraîne-toi à expliquer “à quoi sert une courbe zéro”, et à calculer un taux forward et un DF à la volée.**
